In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data_dir = "/home/x245zhou/verify-c-common/data"
data_arm = pd.read_csv(f"{data_dir}/horn-bv2-lambdas_false_horn-hybrid-array_horn-based-ptr-obj_horn-hybrid-use-arm.brunch.csv")
data_base = pd.read_csv(f"{data_dir}/horn-bv2-lambdas_false_horn-hybrid-array_horn-based-ptr-obj.brunch.csv")

In [8]:
data_arm.fillna(value={'bmc_solve_time': 200, 'bmc_time': 200, "hybrid_row" : 1, "hybrid_thm_skip_store" : 0, "hybrid_arm_skip_store" : 0, "hybrid_arm_skip_memset" : 0, "hybrid_arm_skip_memcpy" : 0})
data_base.fillna(value={'bmc_solve_time': 200, 'bmc_time': 200, "hybrid_row" : 1, "hybrid_thm_skip_store" : 0, "hybrid_arm_skip_store" : 0, "hybrid_arm_skip_memset" : 0, "hybrid_arm_skip_memcpy" : 0})

,job_name,result,bmc_circuit_size,bmc_dag_size,bmc_time,bmc_solve_time,hybrid_smap_new,hybrid_smap_insert,hybrid_smap_hit,hybrid_smap_miss,...,hybrid_smap_skip,hybrid_thm_skip_store,hybrid_arm_skip_store,hybrid_arm_skip_memset,hybrid_arm_skip_memcpy,hybrid_ptr_norm,hybrid_rw_ite,hybrid_row,opsem_simp_time,seahorn_total_time
0,is_power_of_two,True,7.0,26.0,0.02,0.01,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,0.03
1,array_eq_c_str_ignore_case,True,9.0,313.0,0.09,0.09,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.00,0.00,19.0,0.00,0.11
2,add_size_checked,True,5.0,93.0,0.10,0.10,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,0.11
3,add_size_saturating,True,5.0,85.0,0.15,0.15,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,0.16
4,round_up_to_power_of_two,True,5.0,108.0,0.29,0.27,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,hash_iter_delete,True,11.0,1354.0,47.91,47.84,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.00,0.01,519.0,0.03,47.95
224,hash_table_put,True,20.0,4245.0,46.96,46.77,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.01,0.02,1198.0,0.11,47.06
225,hash_table_create,True,24.0,3813.0,56.23,56.04,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.00,0.02,1188.0,0.11,56.33
226,array_list_swap,True,13.0,592.0,73.52,73.49,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.00,0.01,1066.0,0.00,73.54
